# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/qTEA1/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298277235434                   -0.85    5.0         
  2   -8.300163226158       -2.72       -1.25    1.0    100ms
  3   -8.300433163349       -3.57       -1.89    2.1    115ms
  4   -8.300462235344       -4.54       -2.75    2.9    129ms
  5   -8.300463953336       -5.76       -3.08    2.8    134ms
  6   -8.300464285077       -6.48       -3.26    4.0    137ms
  7   -8.300464466015       -6.74       -3.40    8.1    246ms
  8   -8.300464566317       -7.00       -3.55    1.2    138ms
  9   -8.300464626999       -7.22       -3.76    2.8    145ms
 10   -8.300464636790       -8.01       -3.92    1.5    126ms
 11   -8.300464643714       -8.16       -4.25    3.4    173ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67398911856                   -0.70    6.1         
  2   -16.67852554344       -2.34       -1.14    1.0    256ms
  3   -16.67920624009       -3.17       -1.87    3.2    328ms
  4   -16.67927956494       -4.13       -2.76    1.8    272ms
  5   -16.67928601140       -5.19       -3.19    6.6    436ms
  6   -16.67928620319       -6.72       -3.48    2.4    315ms
  7   -16.67928621822       -7.82       -3.94    2.0    265ms
  8   -16.67928622159       -8.47       -4.58    3.0    308ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32671049364                   -0.56    6.8         
  2   -33.33435281584       -2.12       -1.00    1.9    985ms
  3   -33.33591365714       -2.81       -1.72    3.0    1.18s
  4   -33.33613407335       -3.66       -2.61    9.4    1.40s
  5   -33.33679690346       -3.18       -2.24    5.6    1.53s
  6   -33.33682203179       -4.60       -2.29    3.2    998ms
  7   -33.33694129377       -3.92       -3.48    2.0    1.10s
  8   -33.33694268867       -5.86       -3.46    4.5    1.48s
  9   -33.33694312102       -6.36       -3.59    1.0    858ms
 10   -33.33694377632       -6.18       -4.47    2.2    1.08s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298642300528                   -0.85    5.2         
  2   -8.300287758800       -2.78       -1.59    1.0   79.4ms
  3   -8.300442605458       -3.81       -2.74    3.5    116ms
  4   -8.300462426043       -4.70       -3.03    3.1    137ms
  5   -8.300464092943       -5.78       -3.28    1.1   80.8ms
  6   -8.300464611809       -6.28       -3.78    1.5   89.9ms
  7   -8.300464642653       -7.51       -4.53    1.8   93.6ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32609195822                   -0.56    7.2         
  2   -33.33375990473       -2.12       -1.28    1.2    800ms
  3   -33.27323307646   +   -1.22       -1.43    7.5    1.37s
  4   -32.47918790803   +   -0.10       -1.08    4.2    1.31s
  5   -32.39809582740   +   -1.09       -1.05    4.9    1.33s
  6   -33.09449617276       -0.16       -1.34    3.1    1.14s
  7   -33.25375978996       -0.80       -1.57    3.5    1.11s
  8   -33.33661929980       -1.08       -2.48    2.2    962ms
  9   -33.33682231289       -3.69       -2.76    4.4    1.48s
 10   -33.33692399423       -3.99       -3.01    3.1    1.08s
 11   -33.33692617961       -5.66       -3.13    2.4    956ms
 12   -33.33694112710       -4.83      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.